## Data Acquisition

Used Kaggle to obtain data on New York City Weather and New York City collisions. We will be combining these two datasets to comapre collisions with weather. 

Weather dataset: https://www.kaggle.com/datasets/aadimator/nyc-weather-2016-to-2022 

Collisions dataset: https://www.kaggle.com/datasets/amanaayush/nyc-traffic-accidents 

In [25]:
import pandas as pd
import datetime 

In [10]:
collisions = pd.read_csv("NYC_Collisions.csv.zip")
weather = pd.read_csv("NYC_Weather_2016_2022.csv.zip")
print(f"""
    Collisions DataFrame shape: {collisions.shape}
    Weather DataFrame shape: {weather.shape}
      """)


    Collisions DataFrame shape: (238421, 18)
    Weather DataFrame shape: (59760, 10)
      


In [8]:
collisions.head()

,Collision ID,Date,Time,Borough,Street Name,Cross Street,Latitude,Longitude,Contributing Factor,Vehicle Type,Persons Injured,Persons Killed,Pedestrians Injured,Pedestrians Killed,Cyclists Injured,Cyclists Killed,Motorists Injured,Motorists Killed
0,4491746,2021-01-01,20:00:00,Bronx,Bruckner Expressway,NaN,40.83398,-73.826350,Pavement Slippery,Passenger Vehicle,0.0,0,0,0,0,0,0,0
1,4441905,2021-01-01,05:28:00,Brooklyn,Lafayette Avenue,NaN,40.68730,-73.973656,Unspecified,Passenger Vehicle,0.0,0,0,0,0,0,0,0
2,4382769,2021-01-01,06:00:00,Staten Island,West Shore Expressway,NaN,NaN,NaN,Fell Asleep,Passenger Vehicle,0.0,0,0,0,0,0,0,0
3,4380949,2021-01-01,19:30:00,Bronx,Sedgwick Avenue,Vancortlandt Avenue West,40.88270,-73.892730,NaN,Not Reported,0.0,0,0,0,0,0,0,0
4,4380940,2021-01-01,07:40:00,Brooklyn,Cortelyou Road,Mc Donald Avenue,40.63791,-73.978640,Unspecified,Passenger Vehicle,0.0,0,0,0,0,0,0,0


In [11]:
weather.head()

,time,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°)
0,2016-01-01T00:00,7.6,0.0,0.0,69.0,53.0,0.0,72.0,10.0,296.0
1,2016-01-01T01:00,7.5,0.0,0.0,20.0,4.0,0.0,56.0,9.8,287.0
2,2016-01-01T02:00,7.1,0.0,0.0,32.0,3.0,0.0,99.0,9.7,285.0
3,2016-01-01T03:00,6.6,0.0,0.0,35.0,5.0,0.0,100.0,9.2,281.0
4,2016-01-01T04:00,6.3,0.0,0.0,34.0,4.0,0.0,100.0,9.1,279.0


In [22]:
print(f"Collisions columns:")
list(collisions.columns)

Collisions columns:


['Collision ID',
 'Date',
 'Time',
 'Borough',
 'Street Name',
 'Cross Street',
 'Latitude',
 'Longitude',
 'Contributing Factor',
 'Vehicle Type',
 'Persons Injured',
 'Persons Killed',
 'Pedestrians Injured',
 'Pedestrians Killed',
 'Cyclists Injured',
 'Cyclists Killed',
 'Motorists Injured',
 'Motorists Killed']

In [21]:
print(f"Weather columns:")
list(weather.columns)

Weather columns:


['time',
 'temperature_2m (°C)',
 'precipitation (mm)',
 'rain (mm)',
 'cloudcover (%)',
 'cloudcover_low (%)',
 'cloudcover_mid (%)',
 'cloudcover_high (%)',
 'windspeed_10m (km/h)',
 'winddirection_10m (°)']

In [23]:
collisions.dtypes

Collision ID             int64
Date                    object
Time                    object
Borough                 object
Street Name             object
Cross Street            object
Latitude               float64
Longitude              float64
Contributing Factor     object
Vehicle Type            object
Persons Injured        float64
Persons Killed           int64
Pedestrians Injured      int64
Pedestrians Killed       int64
Cyclists Injured         int64
Cyclists Killed          int64
Motorists Injured        int64
Motorists Killed         int64
dtype: object

In [24]:
weather.dtypes

time                      object
temperature_2m (°C)      float64
precipitation (mm)       float64
rain (mm)                float64
cloudcover (%)           float64
cloudcover_low (%)       float64
cloudcover_mid (%)       float64
cloudcover_high (%)      float64
windspeed_10m (km/h)     float64
winddirection_10m (°)    float64
dtype: object

Need to merge the two databases based on date. The *time* value in the *weather* database needs to be convert to date time and then seprate out the date and time. 

## Data Cleaning and Handling Inconsistencies

## Exploratory Data Analysis (EDA)

## Data Preprocessing and Feature Engineering